In [ ]:
!pip install fredapi requests yahoo_fin bs4 tabulate openai==0.28 anthropic google-generativeai rouge_score


import yfinance as yf
import pandas as pd
import random
from fredapi import Fred
import requests
from yahoo_fin import news
from bs4 import BeautifulSoup
from tabulate import tabulate
import random
import openai
import anthropic
#from llama_cpp import Llama
import google.generativeai as genai
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
import re
import random
import nltk
import pandas as pd
from yahoo_fin import news
from datetime import datetime, timedelta
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import time
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

In [ ]:
#Financial News Articles - For Stock (readily available w/ ticker)
from yahoo_fin import news

latest_news = news.get_yf_rss('AAPL')

for i, article in enumerate(latest_news[:10], start=1):
    print(f"{i}. {article['title']}")



In [ ]:
#preprocessing training data
nltk_resources = ["punkt", "stopwords", "wordnet"]

for resource in nltk_resources:
    try:
        nltk.data.find(f"tokenizers/{resource}")
    except LookupError:
        print(f"🔹 Downloading missing NLTK resource: {resource}")
        nltk.download(resource)


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

import re
import random
import nltk
import pandas as pd
from yahoo_fin import news
from datetime import datetime, timedelta
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# 🔹 Download necessary NLTK resources (only needed once)
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

# 🔹 Initialize NLP components
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# 🔹 List of S&P 500 stock tickers (expandable)
sp500_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "FB", "JPM", "V", "DIS",
    "PG", "MA", "NFLX", "INTC", "PYPL", "PFE", "PEP", "KO", "CSCO", "XOM",
    "IBM", "GE", "BA", "GS", "C", "ADBE", "NFLX", "T", "UNH", "MRK",
    "MMM", "LMT", "MDT", "CAT", "MO", "HON", "F", "GM", "MCD", "WMT",
    "NKE", "CVX", "TMO", "ORCL", "DOW", "SO", "DUK", "AEP", "USB", "MET",
    "BK", "TGT", "BLK", "SCHW", "CME", "AON", "SPGI", "MS", "AXP", "CI",
    "ISRG", "ZTS", "PLD", "DE", "ADP", "SYK", "GILD", "GM", "TFC", "CSX",
    "EL", "FDX", "EMR", "ECL", "ADI", "MMC", "WM", "AMT", "SLB", "AIG",
    "CTSH", "DHR", "NSC", "COF", "DG", "FIS", "PGR", "ITW", "CDNS", "APD",
    "AFL", "OXY", "VLO", "HES", "PSX", "TRGP", "MRO", "HAL", "NEM", "BKR"
]

# 🔹 Function to generate a random date between 2011 and 2021
def random_date():
    start_date = datetime(2011, 1, 1)
    end_date = datetime(2021, 12, 31)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# 🔹 Function to clean and preprocess text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = word_tokenize(text)  # Tokenization
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatization
    return " ".join(words)

# 🔹 Fetch and process financial news for at least 100 random ticker-date pairs
news_data = []
attempts = 0

while len(news_data) < 5000 and attempts < 20000:
    attempts += 1
    ticker = random.choice(sp500_tickers)
    date = random_date().strftime('%Y-%m-%d')

    try:

        latest_news = news.get_yf_rss(ticker)

        if latest_news:
            for article in latest_news[:1]:
                title = article["title"]
                link = article["link"]
                cleaned_title = clean_text(title)

                news_data.append({
                    "Ticker": ticker,
                    "Date": date,
                    "Original Title": title,
                    "Cleaned Title": cleaned_title,
                    "Link": link
                })

    except Exception as e:
        print(f"⚠️ Error fetching news for {ticker} on {date}: {e}")

df_news = pd.DataFrame(news_data)

import tabulate
print("\n📊 Processed Historical Financial News:\n")
print(tabulate.tabulate(df_news.head(100), headers="keys", tablefmt="fancy_grid"))

df_news.to_csv("historical_financial_news.csv", index=False)
print("Processed financial news saved to 'historical_financial_news.csv'")

In [ ]:
#preprocessing evaluation data
nltk_resources = ["punkt", "stopwords", "wordnet"]

for resource in nltk_resources:
    try:
        nltk.data.find(f"tokenizers/{resource}")
    except LookupError:
        print(f"🔹 Downloading missing NLTK resource: {resource}")
        nltk.download(resource)


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()



# 🔹 Download necessary NLTK resources (only needed once)
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

# 🔹 Initialize NLP components
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# 🔹 List of S&P 500 stock tickers (expandable)
sp500_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "FB", "JPM", "V", "DIS",
    "PG", "MA", "NFLX", "INTC", "PYPL", "PFE", "PEP", "KO", "CSCO", "XOM",
    "IBM", "GE", "BA", "GS", "C", "ADBE", "NFLX", "T", "UNH", "MRK",
    "MMM", "LMT", "MDT", "CAT", "MO", "HON", "F", "GM", "MCD", "WMT",
    "NKE", "CVX", "TMO", "ORCL", "DOW", "SO", "DUK", "AEP", "USB", "MET",
    "BK", "TGT", "BLK", "SCHW", "CME", "AON", "SPGI", "MS", "AXP", "CI",
    "ISRG", "ZTS", "PLD", "DE", "ADP", "SYK", "GILD", "GM", "TFC", "CSX",
    "EL", "FDX", "EMR", "ECL", "ADI", "MMC", "WM", "AMT", "SLB", "AIG",
    "CTSH", "DHR", "NSC", "COF", "DG", "FIS", "PGR", "ITW", "CDNS", "APD",
    "AFL", "OXY", "VLO", "HES", "PSX", "TRGP", "MRO", "HAL", "NEM", "BKR"
]

# 🔹 Function to generate a random date between 2011 and 2021
def random_date():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 12, 31)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# 🔹 Function to clean and preprocess text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = word_tokenize(text)  # Tokenization
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatization
    return " ".join(words)

# 🔹 Fetch and process financial news for at least 100 random ticker-date pairs
news_data = []
attempts = 0

while len(news_data) < 1500 and attempts < 20000:
    attempts += 1
    ticker = random.choice(sp500_tickers)
    date = random_date().strftime('%Y-%m-%d')

    try:

        latest_news = news.get_yf_rss(ticker)

        if latest_news:
            for article in latest_news[:1]:
                title = article["title"]
                link = article["link"]
                cleaned_title = clean_text(title)

                news_data.append({
                    "Ticker": ticker,
                    "Date": date,
                    "Original Title": title,
                    "Cleaned Title": cleaned_title,
                    "Link": link
                })

    except Exception as e:
        print(f"⚠️ Error fetching news for {ticker} on {date}: {e}")

df_news = pd.DataFrame(news_data)

import tabulate
print("\n📊 Processed Recent Financial News:\n")
print(tabulate.tabulate(df_news.head(100), headers="keys", tablefmt="fancy_grid"))

df_news.to_csv("evaluation_financial_news.csv", index=False)
print("Processed financial news saved to 'evaluation_financial_news.csv'")

In [ ]:
#vader sentiment analysis on training data
nltk.download('vader_lexicon')

df_news = pd.read_csv("historical_financial_news.csv")

sia = SentimentIntensityAnalyzer()

df_news["Sentiment Score"] = df_news["Cleaned Title"].apply(lambda text: sia.polarity_scores(str(text))["compound"])

df_news["Sentiment"] = df_news["Sentiment Score"].apply(
    lambda score: "Positive" if score > 0.05 else "Negative" if score < -0.05 else "Neutral"
)

print("\n📊 VADER Sentiment Analysis Results:\n")
print(df_news[["Ticker", "Date", "Cleaned Title", "Sentiment Score", "Sentiment"]].head(10))

df_news.to_csv("vader_sentiment_analysis_results.csv", index=False)
print("\n✅ Sentiment analysis results saved to 'vader_sentiment_analysis_results.csv'")

In [ ]:
#VADER sentiment analysis on evaluation data

nltk.download('vader_lexicon')

df_eval_news = pd.read_csv("evaluation_financial_news.csv")

sia = SentimentIntensityAnalyzer()

df_eval_news["Sentiment Score"] = df_eval_news["Cleaned Title"].apply(lambda text: sia.polarity_scores(str(text))["compound"])

df_eval_news["Sentiment"] = df_eval_news["Sentiment Score"].apply(
    lambda score: "Positive" if score > 0.05 else "Negative" if score < -0.05 else "Neutral"
)

print("\n📊 VADER Sentiment Analysis Results (2022-2024):\n")
print(df_eval_news[["Ticker", "Date", "Cleaned Title", "Sentiment Score", "Sentiment"]].head(10))

df_eval_news.to_csv("vader_sentiment_analysis_results_2022_2024.csv", index=False)
print("\n✅ Sentiment analysis results saved to 'vader_sentiment_analysis_results_2022_2024.csv'")



In [ ]:
#Training of Anthropic Claude 3.7 using Chain of Thought prompting and Reinforcement Learning
import time
begin = time.time()
df_news = pd.read_csv("historical_financial_news.csv")
df_vader = pd.read_csv("vader_sentiment_analysis_results.csv")

# Initialize Anthropic Claude API
CLAUDE_API_KEY = ""
  # Replace with actual API key
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# Initialize reward tracking
reward_score = 0  # Start with neutral reward score
previous_vader = None
previous_claude = None

# Chain-of-Thought (CoT) Prompting Template
cot_prompt_template = """
You are an AI trained in financial sentiment analysis.
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Do it to 2 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""

# Reinforcement Learning Loop
for index, row in df_news.iterrows():
    article_text = row["Cleaned Title"]

    # Retrieve VADER sentiment score
    vader_sentiment = df_vader.loc[df_vader["Ticker"] == row["Ticker"], "Sentiment Score"].values[0]

    # Update CoT Prompt with Reinforcement Learning Adjustment and Previous Scores
    updated_prompt = cot_prompt_template

    if reward_score < 0.2 and previous_vader is not None and previous_claude is not None:
        updated_prompt += f"\n\n⚠️ Previous predictions have been inaccurate. Pay closer attention to sentiment context."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_claude}."
    elif reward_score > 0.4 and previous_vader is not None and previous_claude is not None:
        updated_prompt += f"\n\n✅ Previous predictions have been accurate. Keep applying the same logic."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_claude}."

    # Format the prompt with the financial news article
    formatted_prompt = updated_prompt.format(article=article_text)

    # Send the CoT prompt to Claude AI
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=500,
        temperature=0.5,
        messages=[{"role": "user", "content": formatted_prompt}]
    )

    # Extract Claude's sentiment prediction using string splitting
    try:
        claude_sentiment = float(response.content[0].text.split("Final Answer: ")[-1].strip())
    except ValueError:
        claude_sentiment = None  # If extraction fails, set to None

    # Reward/Penalty Mechanism (Calculate Absolute Difference)
    if claude_sentiment is not None:
        score_difference = abs(claude_sentiment - vader_sentiment)  # Closer = better
        reward = 1 - score_difference  # Higher reward for smaller difference
        reward_score += reward

        print(f"📰 Article: {article_text}")
        print(f"✅ Claude Score: {claude_sentiment} | VADER Score: {vader_sentiment} | Reward: {reward:.2f}")

        # Store previous scores for next iteration
        previous_vader = vader_sentiment
        previous_claude = claude_sentiment
    else:
        print(f"⚠️ Warning! Could not extract sentiment from Claude's response.")

    # Adjust reward tracking
    print(f"🔹 Current Reward Score: {reward_score:.2f}\n")

print("\n✅ Reinforcement Learning complete! Claude AI has processed all 100 financial news articles with adaptive sentiment refinement.")

df_evaluation = pd.read_csv("evaluation_financial_news.csv")

evaluation_prompt_template = """
You have earlier been provided with a training dataset consisting of equity based financial articles from 2011-2021.
You were asked to perform sentiment analysis on those articles, while being guided as to how accurate your predictions were
Now, you will be prompted with a set of financial articles from 2022-2024, and use the information you gained from the training process to do the following:
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 4 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""

sentiment_results = []
explanation_results = []

for index, row in df_eval_news.iterrows():
    article_text = row["Cleaned Title"]

    # Format the prompt with the financial news article
    formatted_prompt = evaluation_prompt_template.format(article=article_text)

    # Send the CoT prompt to Claude
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=500,
        temperature=0.5,
        messages=[{"role": "user", "content": formatted_prompt}]
    )

    # Extract sentiment prediction and explanation
    response_text = response.content[0].text

    try:
        claude_sentiment = float(response_text.split("Final Answer: ")[-1].strip())
    except ValueError:
        claude_sentiment = None  # If extraction fails, set to None

    claude_explanation = response_text  # Save full explanation

    # Store sentiment results
    sentiment_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Score": claude_sentiment
    })

    # Store sentiment explanation results
    explanation_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Explanation": claude_explanation
    })
    # Print progress
    print(f"✅ Processed {index + 1}/{len(df_eval_news)} articles")
time.sleep(1)
end = time.time()
totalClaudetime = (end-begin)

# Convert results to DataFrames
df_claude_sentiment = pd.DataFrame(sentiment_results)
df_claude_explanations = pd.DataFrame(explanation_results)

# ✅ Save results to CSV for further analysis
df_claude_sentiment.to_csv("claude_sentiment_analysis_results_2022_2024.csv", index=False)
df_claude_explanations.to_csv("claude_sentiment_explanations_2022_2024.csv", index=False)
#Add graph of reward over time - should be logarithmic to prove training is working
print("\n✅ Sentiment analysis results saved to 'claude_sentiment_analysis_results_2022_2024.csv'")
print("✅ Sentiment explanations saved to 'claude_sentiment_explanations_2022_2024.csv'")

In [ ]:
#Training of OpenAI GPT4 using Chain of Thought prompting and Reinforcement Learning
df_news = pd.read_csv("historical_financial_news.csv")
df_vader = pd.read_csv("vader_sentiment_analysis_results.csv")

# Initialize OpenAI GPT-4 API
OPENAI_API_KEY = ''
  # Replace with actual API key
openai.api_key = OPENAI_API_KEY

# Initialize reward tracking
reward_score = 0  # Start with neutral reward score
previous_vader = None
previous_gpt4 = None
begin = time.time()
# Chain-of-Thought (CoT) Prompting Template
cot_prompt_template = """
You are an AI trained in financial sentiment analysis.
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 2 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""

# Reinforcement Learning Loop
for index, row in df_news.iterrows():
    article_text = row["Cleaned Title"]

    # Retrieve VADER sentiment score
    vader_sentiment = df_vader.loc[df_vader["Ticker"] == row["Ticker"], "Sentiment Score"].values[0]

    # Update CoT Prompt with Reinforcement Learning Adjustment and Previous Scores
    updated_prompt = cot_prompt_template

    if reward_score < 0.2 and previous_vader is not None and previous_gpt4 is not None:
        updated_prompt += f"\n\n⚠️ Previous predictions have been inaccurate. Pay closer attention to sentiment context."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_gpt4}."
    elif reward_score > 0.4 and previous_vader is not None and previous_gpt4 is not None:
        updated_prompt += f"\n\n✅ Previous predictions have been accurate. Keep applying the same logic."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_gpt4}."

    # Format the prompt with the financial news article
    formatted_prompt = updated_prompt.format(article=article_text)

    # Send the CoT prompt to GPT-4
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are an expert in financial sentiment analysis."},
                  {"role": "user", "content": formatted_prompt}],
        max_tokens=500,
        temperature=0.5
    )

    # Extract GPT-4 sentiment prediction using string splitting
    try:
        gpt4_sentiment = float(response["choices"][0]["message"]["content"].split("Final Answer: ")[-1].strip())
    except ValueError:
        gpt4_sentiment = None  # If extraction fails, set to None

    #print(gpt4_sentiment)

    #Reward/Penalty Mechanism (Calculate Absolute Difference)
    if gpt4_sentiment is not None:
        score_difference = abs(gpt4_sentiment - vader_sentiment)  # Closer = better
        reward = 1 - score_difference  # Higher reward for smaller difference
        reward_score += reward

        print(f"📰 Article: {article_text}")
        print(f"✅ GPT-4 Score: {gpt4_sentiment} | VADER Score: {vader_sentiment} | Reward: {reward:.2f}")

        # Store previous scores for next iteration
        previous_vader = vader_sentiment
        previous_gpt4 = gpt4_sentiment
    else:
        print(f"⚠️ Warning! Could not extract sentiment from GPT-4's response.")

    # Adjust reward tracking
    print(f"🔹 Current Reward Score: {reward_score:.2f}\n")

print("\n✅ Reinforcement Learning complete! GPT-4 has processed all 100 financial news articles with adaptive sentiment refinement.")

# Evaluation of OpenAI GPT4 after the model has been trained

df_evaluation = pd.read_csv("evaluation_financial_news.csv")

evaluation_prompt_template = """
You have earlier been provided with a training dataset consisting of equity based financial articles from 2011-2021.
You were asked to perform sentiment analysis on those articles, while being guided as to how accurate your predictions were
Now, you will be prompted with a set of financial articles from 2022-2024, and use the information you gained from the training process to do the following:
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 4 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""
sentiment_results = []
explanation_results = []

for index, row in df_eval_news.iterrows():
    article_text = row["Cleaned Title"]

    # Format the prompt with the financial news article
    formatted_prompt = evaluation_prompt_template.format(article=article_text)

    # Send the CoT prompt to GPT-4
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are an expert in financial sentiment analysis."},
                  {"role": "user", "content": formatted_prompt}],
        max_tokens=500,
        temperature=0.5
    )

    # Extract sentiment prediction
    try:
        gpt_sentiment = float(response["choices"][0]["message"]["content"].split("Final Answer: ")[-1].strip())
    except ValueError:
        gpt_sentiment = None  # If extraction fails, set to None

    gpt_explanation = response_text
    # Store results
    sentiment_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Score": gpt_sentiment
    })
    explanation_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Explanation": gpt_explanation
    })

    # Print progress
    print(f"✅ Processed {index + 1}/{len(df_eval_news)} articles")
time.sleep(1)
end = time.time()
totalGPTtime = (end-begin)
df_gpt_sentiment = pd.DataFrame(sentiment_results)
df_gpt_explanations = pd.DataFrame(explanation_results)

df_gpt_sentiment.to_csv("gpt_sentiment_analysis_results_2022_2024.csv", index=False)
df_gpt_explanations.to_csv("gpt_sentiment_explanations_2022_2024.csv", index=False)

print("\n✅ Sentiment analysis results saved to 'gpt_sentiment_analysis_results_2022_2024.csv'")
print("✅ Sentiment explanations saved to 'gpt_sentiment_explanations_2022_2024.csv'")

In [ ]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN=userdata.get('test_token')

if HF_TOKEN:
    login(HF_TOKEN)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token is not set. Please save the token first.")

In [ ]:
#Training Meta LLaMa - only do this if it works
'''
# 🔹 Load the 100 preprocessed financial news articles and VADER sentiment results
df_news = pd.read_csv("historical_financial_news.csv")
df_vader = pd.read_csv("vader_sentiment_analysis_results.csv")

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
begin = time.time()
# 🔹 Initialize reward tracking
reward_score = 0  # Start with neutral reward score
previous_vader = None
previous_llama = None

# 🔹 Chain-of-Thought (CoT) Prompting Template
cot_prompt_template = """
You are an AI trained in financial sentiment analysis.
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 2 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""

# 🔹 Reinforcement Learning Loop
for index, row in df_news.iterrows():
    article_text = row["Cleaned Title"]

    # Retrieve VADER sentiment score
    vader_sentiment = df_vader.loc[df_vader["Ticker"] == row["Ticker"], "Sentiment Score"].values[0]

    # 🔹 Update CoT Prompt with Reinforcement Learning Adjustment and Previous Scores
    updated_prompt = cot_prompt_template

    if reward_score < 0.2 and previous_vader is not None and previous_llama is not None:
        updated_prompt += f"\n\n⚠️ Previous predictions have been inaccurate. Pay closer attention to sentiment context."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_llama}."
    elif reward_score > 0.4 and previous_vader is not None and previous_llama is not None:
        updated_prompt += f"\n\n✅ Previous predictions have been accurate. Keep applying the same logic."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_llama}."

    formatted_prompt = updated_prompt.format(article=article_text)

    # Generate response from LLaMA
    response = pipe(formatted_prompt, max_new_tokens=200)
    response_text = response[0]["generated_text"]

    # Extract LLaMA sentiment prediction
    match = re.search(r"Final Answer: (-?\d+\.\d+)", response_text)
    llama_sentiment = float(match.group(1)) if match else None

    # 🔹 Reward/Penalty Mechanism (Calculate Absolute Difference)
    if llama_sentiment is not None:
        score_difference = abs(llama_sentiment - vader_sentiment)  # Closer = better
        reward = 1 - score_difference  # Higher reward for smaller difference
        reward_score += reward

        print(f"📰 Article: {article_text}")
        print(f"✅ LLaMA Score: {llama_sentiment} | VADER Score: {vader_sentiment} | Reward: {reward:.2f}")

        # Store previous scores for next iteration
        previous_vader = vader_sentiment
        previous_llama = llama_sentiment
    else:
        print(f"⚠️ Warning! Could not extract sentiment from LLaMA's response.")

    # Adjust reward tracking
    print(f"🔹 Current Reward Score: {reward_score:.2f}\n")

print("\n✅ Reinforcement Learning complete! LLaMA has processed all 100 financial news articles with adaptive sentiment refinement.")

df_eval_news = pd.read_csv("evaluation_financial_news.csv")

evaluation_prompt_template = """
You have earlier been provided with a training dataset consisting of equity based financial articles from 2011-2021.
You were asked to perform sentiment analysis on those articles, while being guided as to how accurate your predictions were
Now, you will be prompted with a set of financial articles from 2022-2024, and use the information you gained from the training process to do the following:
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 4 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""


sentiment_results = []

for index, row in df_eval_news.iterrows():
    article_text = row["Cleaned Title"]

    # Format the prompt with the financial news article
    formatted_prompt = evaluation_prompt_template.format(article=article_text)

    response = pipe(formatted_prompt, max_new_tokens=200)
    response_text = response[0]["generated_text"]

    # Extract LLaMA sentiment prediction
    match = re.search(r"Final Answer: (-?\d+\.\d+)", response_text)
    llama_sentiment = float(match.group(1)) if match else None

    # Extract LLaMA sentiment prediction using string splitting
    try:
        llama_sentiment = float(response_text.split("Final Answer: ")[-1].strip())
    except ValueError:
        llama_sentiment = None  # If extraction fails, set to None

    # Store results
    sentiment_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Score": llama_sentiment
    })

    # Print progress
    print(f"✅ Processed {index + 1}/{len(df_eval_news)} articles")
time.sleep(1)
end = time.time()
totalLlamatime = (end-begin)
# Convert results to DataFrame
df_llama_sentiment = pd.DataFrame(sentiment_results)

# ✅ Save results to CSV for further analysis
df_llama_sentiment.to_csv("llama_sentiment_analysis_results_2022_2024.csv", index=False)

print("\n✅ Sentiment analysis results saved to 'llama_sentiment_analysis_results_2022_2024.csv'")
'''

In [ ]:

#Training of Google Gemini using Chain of Thought and Reinforcement Learning - only pay and use if meta LLaMa doesn't work

df_news = pd.read_csv("historical_financial_news.csv")
df_vader = pd.read_csv("vader_sentiment_analysis_results.csv")

# Initialize Google Gemini API

GEMINI_API_KEY = ""  # Replace with actual API key
genai.configure(api_key=GEMINI_API_KEY)
begin = time.time()

# Load Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")

# Initialize reward tracking
reward_score = 0  # Start with neutral reward score
previous_vader = None
previous_gemini = None

# Chain-of-Thought (CoT) Prompting Template
cot_prompt_template = """
You are an AI trained in financial sentiment analysis.
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.
Article: "{article}"

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 2 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""
reward_scores_list = []
# Reinforcement Learning Loop
for index, row in df_news.iterrows():
    article_text = row["Cleaned Title"]

    # Retrieve VADER sentiment score
    vader_sentiment = df_vader.loc[df_vader["Ticker"] == row["Ticker"], "Sentiment Score"].values[0]

    # Update CoT Prompt with Reinforcement Learning Adjustment and Previous Scores
    updated_prompt = cot_prompt_template

    if reward_score < 0.2 and previous_vader is not None and previous_gemini is not None:
        updated_prompt += f"\n\n⚠️ Previous predictions have been inaccurate. Pay closer attention to sentiment context."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_gemini}."
    elif reward_score > 0.4 and previous_vader is not None and previous_gemini is not None:
        updated_prompt += f"\n\n✅ Previous predictions have been accurate. Keep applying the same logic."
        updated_prompt += f" The last article had a VADER sentiment score of {previous_vader} and you provided {previous_gemini}."
    print("Article text = " + article_text)
    # Format the prompt with the financial news article
    formatted_prompt = updated_prompt.format(article=article_text)

    # Generate response from Gemini
    response = model.generate_content(formatted_prompt)
    #print(response)
    # Extract Gemini sentiment prediction using string splitting
    try:
        gemini_sentiment = float(response.text.split("Final Answer: ")[-1].strip())
    except ValueError:
        gemini_sentiment = None  # If extraction fails, set to None

    # Reward/Penalty Mechanism (Calculate Absolute Difference)
    if gemini_sentiment is not None:
        score_difference = abs(gemini_sentiment - vader_sentiment)  # Closer = better
        reward = 1 - score_difference  # Higher reward for smaller difference
        reward_score += reward

        reward_scores_list.append({
        "Ticker": ticker,
        "Date": date,
        "Reward Score": reward_score
        })

        print(f"📰 Article: {article_text}")
        print(f"✅ Gemini Score: {gemini_sentiment} | VADER Score: {vader_sentiment} | Reward: {reward:.2f}")

        # Store previous scores for next iteration
        previous_vader = vader_sentiment
        previous_gemini = gemini_sentiment
    else:
        print(f"⚠️ Warning! Could not extract sentiment from Gemini's response.")

    # Adjust reward tracking
    print(f"Current Reward Score: {reward_score:.2f}\n")

print("\n✅ Reinforcement Learning complete! Gemini has processed all 100 financial news articles with adaptive sentiment refinement.")


df_eval_news = pd.read_csv("evaluation_financial_news.csv")


evaluation_prompt_template = """
You have earlier been provided with a training dataset consisting of equity based financial articles from 2011-2021.
You were asked to perform sentiment analysis on those articles, while being guided as to how accurate your predictions were
Now, you will be prompted with a set of financial articles from 2022-2024, and use the information you gained from the training process to do the following:
Analyze the sentiment of the following financial news article step by step. Limit your answer in total to 500 characters.
Article: "{article}"

1. **Extract Key Phrases**: Identify important financial terms, events, and entities.
2. **Determine Sentiment Score**: Provide a sentiment score between -1 (Negative), 0 (Neutral), and +1 (Positive). Please do this to 4 decimal places
3. **Explain Sentiment**: Justify the assigned sentiment score based on the financial impact.

Example Sentiment Analysis:

Article: "{article}"
---
Step 1: Key Phrases → [extracted key phrases]
Step 2: Sentiment Score → [Numeric value between -1 and 1]
Step 3: Explanation → [detailed reasoning]

Final Answer: [Numeric Sentiment Score]
"""
sentiment_results = []
explanation_results = []

for index, row in df_eval_news.iterrows():
    article_text = row["Cleaned Title"]

    # Format the prompt with the financial news article
    formatted_prompt = evaluation_prompt_template.format(article=article_text)

    # 🔹 Generate response from Gemini
    response = model.generate_content(formatted_prompt)

    # Extract sentiment prediction and explanation
    response_text = response.text

    try:
        gemini_sentiment = float(response_text.split("Final Answer: ")[-1].strip())
    except ValueError:
        gemini_sentiment = None  # If extraction fails, set to None

    gemini_explanation = response_text  # Save full explanation
    print(gemini_explanation)
    # Store sentiment results
    sentiment_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Score": gemini_sentiment
    })

    # Store sentiment explanation results
    explanation_results.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "Original Title": row["Original Title"],
        "Cleaned Title": row["Cleaned Title"],
        "Sentiment Explanation": gemini_explanation
    })

    # Print progress
    print(f"✅ Processed {index + 1}/{len(df_eval_news)} articles")
time.sleep(1)
end = time.time()
totalGeminitime = (end-begin)
# Convert results to DataFrames
df_gemini_sentiment = pd.DataFrame(sentiment_results)
df_gemini_explanations = pd.DataFrame(explanation_results)
df_rewards = pd.DataFrame(reward_scores_list)
df_rewards.to_csv("gemini_reward_scores_training.csv", index=False)
print("\n✅ Reward scores saved to 'gemini_reward_scores_training.csv'")
# ✅ Save results to CSV for further analysis
df_gemini_sentiment.to_csv("gemini_sentiment_analysis_results_2022_2024.csv", index=False)
df_gemini_explanations.to_csv("gemini_sentiment_explanations_2022_2024.csv", index=False)

print("\n✅ Sentiment analysis results saved to 'gemini_sentiment_analysis_results_2022_2024.csv'")
print("✅ Sentiment explanations saved to 'gemini_sentiment_explanations_2022_2024.csv'")

In [ ]:
import matplotlib.pyplot as plt
from rouge_score import rouge_scorer

# 🔹 Load sentiment explanation datasets
df_claude = pd.read_csv("claude_sentiment_explanations_2022_2024.csv")
df_gpt = pd.read_csv("gpt_sentiment_explanations_2022_2024.csv")
df_gemini = pd.read_csv("gemini_sentiment_explanations_2022_2024.csv")

# 🔹 Merge datasets by Ticker and Date to align explanations
df_merged = pd.merge(df_claude, df_gpt, on=["Ticker", "Date", "Original Title", "Cleaned Title"], suffixes=("_claude", "_gpt"))
df_merged = pd.merge(df_merged, df_gemini, on=["Ticker", "Date", "Original Title", "Cleaned Title"])
df_merged.rename(columns={"Sentiment Explanation": "Sentiment Explanation_gemini"}, inplace=True)

# 🔹 Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# 🔹 Compute ROUGE F1 scores for each explanation pair
rouge_scores = []

for index, row in df_merged.iterrows():
    claude_text = row["Sentiment Explanation_claude"]
    gpt_text = row["Sentiment Explanation_gpt"]
    gemini_text = row["Sentiment Explanation_gemini"]

    # Compute ROUGE scores
    scores_claude_gpt = scorer.score(claude_text, gpt_text)
    scores_claude_gemini = scorer.score(claude_text, gemini_text)
    scores_gpt_gemini = scorer.score(gpt_text, gemini_text)

    rouge_scores.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "ROUGE-1 Claude vs GPT": scores_claude_gpt["rouge1"].fmeasure,
        "ROUGE-2 Claude vs GPT": scores_claude_gpt["rouge2"].fmeasure,
        "ROUGE-L Claude vs GPT": scores_claude_gpt["rougeL"].fmeasure,
        "ROUGE-1 Claude vs Gemini": scores_claude_gemini["rouge1"].fmeasure,
        "ROUGE-2 Claude vs Gemini": scores_claude_gemini["rouge2"].fmeasure,
        "ROUGE-L Claude vs Gemini": scores_claude_gemini["rougeL"].fmeasure,
        "ROUGE-1 GPT vs Gemini": scores_gpt_gemini["rouge1"].fmeasure,
        "ROUGE-2 GPT vs Gemini": scores_gpt_gemini["rouge2"].fmeasure,
        "ROUGE-L GPT vs Gemini": scores_gpt_gemini["rougeL"].fmeasure,
    })

# 🔹 Convert to DataFrame
df_rouge_results = pd.DataFrame(rouge_scores)

# ✅ Save results to CSV
df_rouge_results.to_csv("rouge_f1_comparison_claude_gpt_gemini.csv", index=False)

print("\n✅ ROUGE F1 comparison results saved to 'rouge_f1_comparison_claude_gpt_gemini.csv'")

# 📌 Display summary statistics
print("\n📌 ROUGE F1 Score Summary:")
print(df_rouge_results.describe())

# 🔹 Plot ROUGE-1 Differences
plt.figure(figsize=(12, 5))
plt.plot(df_rouge_results["ROUGE-1 Claude vs GPT"], label="ROUGE-1 Claude vs GPT", linestyle='-', marker='o', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-1 Claude vs Gemini"], label="ROUGE-1 Claude vs Gemini", linestyle='-', marker='s', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-1 GPT vs Gemini"], label="ROUGE-1 GPT vs Gemini", linestyle='-', marker='d', alpha=0.7)

plt.xlabel("Articles")
plt.ylabel("ROUGE-1 F1 Score")
plt.title("ROUGE-1 F1 Score Differences Between Claude, GPT-4, and Gemini")
plt.legend()
plt.grid(True)
plt.show()

# 🔹 Plot ROUGE-2 Differences
plt.figure(figsize=(12, 5))
plt.plot(df_rouge_results["ROUGE-2 Claude vs GPT"], label="ROUGE-2 Claude vs GPT", linestyle='-', marker='o', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-2 Claude vs Gemini"], label="ROUGE-2 Claude vs Gemini", linestyle='-', marker='s', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-2 GPT vs Gemini"], label="ROUGE-2 GPT vs Gemini", linestyle='-', marker='d', alpha=0.7)

plt.xlabel("Articles")
plt.ylabel("ROUGE-2 F1 Score")
plt.title("ROUGE-2 F1 Score Differences Between Claude, GPT-4, and Gemini")
plt.legend()
plt.grid(True)
plt.show()

# 🔹 Plot ROUGE-L Differences
plt.figure(figsize=(12, 5))
plt.plot(df_rouge_results["ROUGE-L Claude vs GPT"], label="ROUGE-L Claude vs GPT", linestyle='-', marker='o', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-L Claude vs Gemini"], label="ROUGE-L Claude vs Gemini", linestyle='-', marker='s', alpha=0.7)
plt.plot(df_rouge_results["ROUGE-L GPT vs Gemini"], label="ROUGE-L GPT vs Gemini", linestyle='-', marker='d', alpha=0.7)

plt.xlabel("Articles")
plt.ylabel("ROUGE-L F1 Score")
plt.title("ROUGE-L F1 Score Differences Between Claude, GPT-4, and Gemini")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from rouge_score import rouge_scorer

In [ ]:
# 🔹 Load sentiment analysis results for Claude and VADER
df_vader = pd.read_csv("vader_sentiment_analysis_results_2022_2024.csv")
df_claude = pd.read_csv("claude_sentiment_analysis_results_2022_2024.csv")
df_claude_explanations = pd.read_csv("claude_sentiment_explanations_2022_2024.csv")

# 🔹 Merge datasets on Ticker and Date to align sentiment scores
df_merged = df_vader.merge(df_claude, on=["Ticker", "Date", "Original Title", "Cleaned Title"], suffixes=("_vader", "_claude"))

# 🔹 Extract sentiment scores
vader_scores = df_merged["Sentiment Score_vader"].values
claude_scores = df_merged["Sentiment Score_claude"].values

# 🔹 Replace NaNs and infinite values with finite numbers
vader_scores = np.nan_to_num(vader_scores, nan=0.0, posinf=1.0, neginf=-1.0)
claude_scores = np.nan_to_num(claude_scores, nan=0.0, posinf=1.0, neginf=-1.0)

# 🔹 Compute Pearson Correlation Coefficient safely
if len(vader_scores) > 1 and len(claude_scores) > 1:
    pearson_claude, _ = pearsonr(vader_scores, claude_scores)
else:
    pearson_claude = np.nan  # Assign NaN if correlation cannot be computed

# 🔹 Compute Mean Squared Error (MSE)
mse_claude = mean_squared_error(vader_scores, claude_scores)

# 🔹 Compute ROUGE F1 scores for explanations
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = []

for index, row in df_claude_explanations.iterrows():
    reference_text = row["Sentiment Explanation"]
    candidate_text = row["Sentiment Explanation"]  # Compare Claude to itself for structure

    scores = scorer.score(reference_text, candidate_text)

    rouge_scores.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "ROUGE-1 F1": scores["rouge1"].fmeasure if scores["rouge1"].fmeasure is not None else 0.0,
        "ROUGE-2 F1": scores["rouge2"].fmeasure if scores["rouge2"].fmeasure is not None else 0.0,
        "ROUGE-L F1": scores["rougeL"].fmeasure if scores["rougeL"].fmeasure is not None else 0.0,
    })

df_rouge_results = pd.DataFrame(rouge_scores)

# 🔹 Replace NaNs/Infs in ROUGE scores
df_rouge_results.fillna(0, inplace=True)

# 🔹 Calculate average ROUGE F1 scores
rouge1_f1_avg = df_rouge_results["ROUGE-1 F1"].mean()
rouge2_f1_avg = df_rouge_results["ROUGE-2 F1"].mean()
rougeL_f1_avg = df_rouge_results["ROUGE-L F1"].mean()

# 📊 Store evaluation results
evaluation_results_claude = pd.DataFrame({
    "Metric": ["MSE", "Pearson Correlation", "ROUGE-1 F1", "ROUGE-2 F1", "ROUGE-L F1"],
    "Claude vs VADER": [mse_claude, pearson_claude, rouge1_f1_avg, rouge2_f1_avg, rougeL_f1_avg]
})

print("MSE: " + str(mse_claude))
print("Pearson: " + str(pearson_claude))

# ✅ Save results to CSV
evaluation_results_claude.to_csv("evaluation_metrics_claude_vs_vader.csv", index=False)
print("\n✅ Evaluation results saved to 'evaluation_metrics_claude_vs_vader.csv'")

# 🔹 Plot comparison
plt.figure(figsize=(8, 5))
metrics = evaluation_results_claude["Metric"]
values = evaluation_results_claude["Claude vs VADER"]

plt.barh(metrics, values, alpha=0.7, color="blue", label="Claude vs VADER")
plt.xlabel("Score")
plt.ylabel("Metric")
plt.title("Claude vs VADER: MSE, Pearson R, ROUGE F1 (Handling NaNs/Infs)")
plt.legend()
plt.grid(True)

# 📊 Show plot
plt.show()

In [ ]:


# 🔹 Load sentiment analysis results for GPT-4 and VADER
df_vader = pd.read_csv("vader_sentiment_analysis_results_2022_2024.csv")
df_gpt = pd.read_csv("gpt_sentiment_analysis_results_2022_2024.csv")
df_gpt_explanations = pd.read_csv("gpt_sentiment_explanations_2022_2024.csv")

# 🔹 Merge datasets on Ticker and Date to align sentiment scores
df_merged = df_vader.merge(df_gpt, on=["Ticker", "Date", "Original Title", "Cleaned Title"], suffixes=("_vader", "_gpt"))

# 🔹 Extract sentiment scores
vader_scores = df_merged["Sentiment Score_vader"].values
gpt_scores = df_merged["Sentiment Score_gpt"].values

# 🔹 Replace NaNs and infinite values with finite numbers
vader_scores = np.nan_to_num(vader_scores, nan=0.0, posinf=1.0, neginf=-1.0)
gpt_scores = np.nan_to_num(gpt_scores, nan=0.0, posinf=1.0, neginf=-1.0)

# 🔹 Compute Pearson Correlation Coefficient safely
if len(vader_scores) > 1 and len(gpt_scores) > 1:
    pearson_gpt, _ = pearsonr(vader_scores, gpt_scores)
else:
    pearson_gpt = np.nan  # Assign NaN if correlation cannot be computed

# 🔹 Compute Mean Squared Error (MSE)
mse_gpt = mean_squared_error(vader_scores, gpt_scores)

# 🔹 Compute ROUGE F1 scores for explanations
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = []

for index, row in df_gpt_explanations.iterrows():
    reference_text = row["Sentiment Explanation"]
    candidate_text = row["Sentiment Explanation"]  # Compare GPT to itself for structure

    scores = scorer.score(reference_text, candidate_text)

    rouge_scores.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "ROUGE-1 F1": scores["rouge1"].fmeasure if scores["rouge1"].fmeasure is not None else 0.0,
        "ROUGE-2 F1": scores["rouge2"].fmeasure if scores["rouge2"].fmeasure is not None else 0.0,
        "ROUGE-L F1": scores["rougeL"].fmeasure if scores["rougeL"].fmeasure is not None else 0.0,
    })

df_rouge_results = pd.DataFrame(rouge_scores)

# 🔹 Replace NaNs/Infs in ROUGE scores
df_rouge_results.fillna(0, inplace=True)

# 🔹 Calculate average ROUGE F1 scores
rouge1_f1_avg = df_rouge_results["ROUGE-1 F1"].mean()
rouge2_f1_avg = df_rouge_results["ROUGE-2 F1"].mean()
rougeL_f1_avg = df_rouge_results["ROUGE-L F1"].mean()

# 📊 Store evaluation results
evaluation_results_gpt = pd.DataFrame({
    "Metric": ["MSE", "Pearson Correlation", "ROUGE-1 F1", "ROUGE-2 F1", "ROUGE-L F1"],
    "GPT-4 vs VADER": [mse_gpt, pearson_gpt, rouge1_f1_avg, rouge2_f1_avg, rougeL_f1_avg]
})

print("MSE: " + str(mse_gpt))
print("Pearson: " + str(pearson_gpt))

# ✅ Save results to CSV
evaluation_results_gpt.to_csv("evaluation_metrics_gpt_vs_vader.csv", index=False)
print("\n✅ Evaluation results saved to 'evaluation_metrics_gpt_vs_vader.csv'")

# 🔹 Plot comparison
plt.figure(figsize=(8, 5))
metrics = evaluation_results_gpt["Metric"]
values = evaluation_results_gpt["GPT-4 vs VADER"]

plt.barh(metrics, values, alpha=0.7, color="green", label="GPT-4 vs VADER")
plt.xlabel("Score")
plt.ylabel("Metric")
plt.title("GPT-4 vs VADER: MSE, Pearson R, ROUGE F1 (Handling NaNs/Infs)")
plt.legend()
plt.grid(True)

# 📊 Show plot
plt.show()

In [ ]:
from scipy.stats import pearsonr
from rouge_score import rouge_scorer

# 🔹 Load sentiment analysis results for Gemini and VADER
df_vader = pd.read_csv("vader_sentiment_analysis_results_2022_2024.csv")
df_gemini = pd.read_csv("gemini_sentiment_analysis_results_2022_2024.csv")
df_gemini_explanations = pd.read_csv("gemini_sentiment_explanations_2022_2024.csv")

# 🔹 Merge datasets on Ticker and Date to align sentiment scores
df_merged = df_vader.merge(df_gemini, on=["Ticker", "Date", "Original Title", "Cleaned Title"], suffixes=("_vader", "_gemini"))

# 🔹 Extract sentiment scores
vader_scores = df_merged["Sentiment Score_vader"]
gemini_scores = df_merged["Sentiment Score_gemini"]

# 🔹 Handle NaN values: Drop rows where either VADER or Gemini score is NaN
valid_indices = ~np.isnan(gemini_scores) & ~np.isnan(vader_scores)
vader_scores = vader_scores[valid_indices]
gemini_scores = gemini_scores[valid_indices]

# 🔹 Compute Pearson Correlation Coefficient
if len(vader_scores) > 1 and len(gemini_scores) > 1:
    pearson_gemini, _ = pearsonr(vader_scores, gemini_scores)
else:
    pearson_gemini = np.nan  # Assign NaN if correlation cannot be computed

# 🔹 Compute Mean Squared Error (MSE)
mse_gemini = mean_squared_error(vader_scores, gemini_scores)

# 🔹 Compute ROUGE F1 scores for explanations
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = []

for index, row in df_gemini_explanations.iterrows():
    reference_text = row["Sentiment Explanation"]
    candidate_text = row["Sentiment Explanation"]  # Compare Gemini to itself for structure

    scores = scorer.score(reference_text, candidate_text)

    rouge_scores.append({
        "Ticker": row["Ticker"],
        "Date": row["Date"],
        "ROUGE-1 F1": scores["rouge1"].fmeasure,
        "ROUGE-2 F1": scores["rouge2"].fmeasure,
        "ROUGE-L F1": scores["rougeL"].fmeasure
    })

df_rouge_results = pd.DataFrame(rouge_scores)

# 🔹 Calculate average ROUGE F1 scores
rouge1_f1_avg = df_rouge_results["ROUGE-1 F1"].mean()
rouge2_f1_avg = df_rouge_results["ROUGE-2 F1"].mean()
rougeL_f1_avg = df_rouge_results["ROUGE-L F1"].mean()

# 📊 Store evaluation results
evaluation_results_gemini = pd.DataFrame({
    "Metric": ["MSE", "Pearson Correlation", "ROUGE-1 F1", "ROUGE-2 F1", "ROUGE-L F1"],
    "Gemini vs VADER": [mse_gemini, pearson_gemini, rouge1_f1_avg, rouge2_f1_avg, rougeL_f1_avg]
})
print("MSE: " + str(mse_gemini))
print("Pearson: " + str(pearson_gemini))

# ✅ Save results to CSV
evaluation_results_gemini.to_csv("evaluation_metrics_gemini_vs_vader.csv", index=False)
print("\n✅ Evaluation results saved to 'evaluation_metrics_gemini_vs_vader.csv'")

# 🔹 Plot comparison
plt.figure(figsize=(8, 5))
metrics = evaluation_results_gemini["Metric"]
values = evaluation_results_gemini["Gemini vs VADER"]

plt.barh(metrics, values, alpha=0.7, color="purple", label="Gemini vs VADER")
plt.xlabel("Score")
plt.ylabel("Metric")
plt.title("Gemini vs VADER: MSE, Pearson R, ROUGE F1 (Handling NaN)")
plt.legend()
plt.grid(True)

# 📊 Show plot
plt.show()

In [ ]:

# 🔹 Load the reward scores CSV
df_rewards = pd.read_csv("gemini_reward_scores_training.csv")

# 🔹 Generate X-axis values (Number of closing prices processed)
num_prices_processed = list(range(1, len(df_rewards) + 1))  # Sequential numbering

# 🔹 Extract Reward Scores
reward_scores = df_rewards["Reward Score"]

# 🔹 Plot the Reward Score Trend
plt.figure(figsize=(12, 6))
plt.plot(num_prices_processed, reward_scores, marker="o", linestyle="-", color="blue", label="Reward Score")

# 🔹 Graph Labels & Customization
plt.xlabel("Number of Closing Prices Processed")
plt.ylabel("Reward Score")
plt.title("Gemini Training Reward Score vs. Number of Articles Processed")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)

# 📈 Show the plot
plt.show()

In [ ]:
print("Claude Time taken = " + str(totalClaudetime))
print("GPT Time taken = " + str(totalGPTtime))
print("Gemini Time taken = " + str(totalGeminitime))

models = ["Claude", "GPT-4", "Gemini"]
time_taken = [totalClaudetime, totalGPTtime, totalGeminitime]

# Create bar plot
plt.figure(figsize=(8, 5))
plt.bar(models, time_taken, color=['blue', 'red', 'green'])

# Labels and title
plt.xlabel("Model")
plt.ylabel("Time Taken (seconds)")
plt.title("Comparison of Time Taken for Claude, GPT-4, and Gemini")

# Show grid and plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()